<a href="https://colab.research.google.com/github/ryan-saloma/teaching-python/blob/main/Project_1_(Taylor_Swift_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Libraries and Data

In [1]:
import pandas as pd

data = pd.read_csv('taylor_swift_spotify.csv')

In [7]:
# Explore the data

# Look at the first few rows
data.head()

# Look at the column names, number of rows, and data types
# Check for missing values
# data.info()

# Generate basic summary stats
# data.describe()

# Check for duplicates
data['name'].value_counts()



name
illicit affairs                         3
22                                      3
epiphany                                3
betty                                   3
peace                                   3
                                       ..
I Forgot That You Existed               1
Cruel Summer                            1
Lover                                   1
The Man                                 1
Teardrops on My Guitar - Pop Version    1
Name: count, Length: 360, dtype: int64